# Superpixels Explorer

Interactively compute and visualize superpixel overlays for `original.png` and `edited.png`.

- Place the images in the same folder as this notebook (or anywhere under it).
- Use the sliders/toggles to tune SLIC parameters and visualization.
- View side-by-side results for both images.

Notes:
- This uses `skimage.segmentation.slic`, `skimage.segmentation.mark_boundaries`, and `skimage.color.label2rgb`.
- If packages are missing, the next cell attempts to install them automatically.

In [7]:
# Imports, setup, and image loading
# If you are missing dependencies, uncomment the pip line below.
# %pip install scikit-image ipywidgets pillow matplotlib

from pathlib import Path
import numpy as np
from skimage.segmentation import slic, mark_boundaries
from skimage import img_as_float
from skimage.io import imread
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, FloatSlider, FloatLogSlider, Checkbox, Dropdown, ToggleButtons, interactive_output, HBox, VBox
from IPython.display import display

IMG_DIR = Path('.')
ORIG_PATH = IMG_DIR / 'original.png'
EDIT_PATH = IMG_DIR / 'edited.png'

def safe_read(path: Path):
    if not path.exists():
        raise FileNotFoundError(f"Image not found: {path}")
    img = imread(path)
    if img.ndim == 2:  # grayscale to RGB
        img = np.stack([img]*3, axis=-1)
    return img_as_float(img)

try:
    original_img = safe_read(ORIG_PATH)
    edited_img = safe_read(EDIT_PATH)
    print('Loaded images:')
    print(' original:', original_img.shape, original_img.dtype)
    print(' edited  :', edited_img.shape, edited_img.dtype)
except FileNotFoundError as e:
    print(e)
    print('')
    print('Please ensure both `original.png` and `edited.png` are present in the notebook directory.')

Loaded images:
 original: (1024, 1024, 4) float64
 edited  : (1024, 1024, 4) float64


In [9]:
# Superpixel computation and interactive UI
import warnings
warnings.filterwarnings('ignore')

from skimage.color import label2rgb
from skimage.segmentation import find_boundaries

# Controls
n_segments_slider = IntSlider(value=400, min=50, max=5000, step=10, description='n_segments', continuous_update=False)
compactness_slider = FloatLogSlider(value=10.0, base=10, min=-2, max=2, step=0.05, description='compactness', continuous_update=False)
sigma_slider = FloatSlider(value=0.0, min=0.0, max=5.0, step=0.1, description='sigma', continuous_update=False)
enforce_checkbox = Checkbox(value=True, description='enforce_connectivity')
start_label_selector = ToggleButtons(options=[0,1], value=1, description='start_label')

boundary_mode_dropdown = Dropdown(options=['thick','inner','outer','subpixel'], value='thick', description='boundary_mode')
boundary_color_dropdown = Dropdown(options=['yellow','red','blue','green','white','black'], value='yellow', description='boundary_color')
fill_checkbox = Checkbox(value=False, description='fill with avg')
fill_alpha_slider = FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='fill_alpha')

controls = VBox([
    HBox([n_segments_slider, compactness_slider, sigma_slider]),
    HBox([enforce_checkbox, start_label_selector]),
    HBox([boundary_mode_dropdown, boundary_color_dropdown]),
    HBox([fill_checkbox, fill_alpha_slider])
])

BOUNDARY_COLORS = {
    'yellow': (1.0, 1.0, 0.0),
    'red': (1.0, 0.0, 0.0),
    'blue': (0.0, 0.4, 1.0),
    'green': (0.0, 1.0, 0.0),
    'white': (1.0, 1.0, 1.0),
    'black': (0.0, 0.0, 0.0),
}

IMAGE_FILES = {
    'original': ORIG_PATH,
    'edited': EDIT_PATH,
}

IMAGE_CACHE = {}
for k, p in IMAGE_FILES.items():
    if p.exists():
        IMAGE_CACHE[k] = safe_read(p)

if len(IMAGE_CACHE) < 2:
    print('Warning: Not all images found. Found keys:', list(IMAGE_CACHE.keys()))

# Core function
def compute_and_show(n_segments, compactness, sigma, enforce_connectivity, start_label, boundary_mode, boundary_color, fill, fill_alpha):
    overlay_imgs = []
    boundary_imgs = []
    titles = []
    boundary_rgb_color = BOUNDARY_COLORS.get(boundary_color, (1.0,1.0,0.0))
    for key in ['original','edited']:
        if key not in IMAGE_CACHE:
            continue
        img = IMAGE_CACHE[key]
        # Handle RGBA (4 channels) by stripping alpha for SLIC but keep it for display
        if img.ndim == 3 and img.shape[-1] == 4:
            rgb_for_slic = img[..., :3]
            alpha_channel = img[..., 3:4]
        else:
            rgb_for_slic = img
            alpha_channel = None
        # Compute SLIC (only convert to Lab when 3-channel RGB)
        convert_to_lab = (rgb_for_slic.ndim == 3 and rgb_for_slic.shape[-1] == 3)
        segments = slic(
            rgb_for_slic,
            n_segments=int(n_segments),
            compactness=float(compactness),
            sigma=float(sigma),
            start_label=int(start_label),
            enforce_connectivity=bool(enforce_connectivity),
            convert2lab=convert_to_lab,
            channel_axis=-1,
        )
        base = rgb_for_slic.copy()
        if fill:
            # Average color per superpixel then blend with original (use RGB for averaging)
            avg = label2rgb(segments, image=rgb_for_slic, kind='avg')
            base = (1.0 - float(fill_alpha)) * rgb_for_slic + float(fill_alpha) * avg
        with_boundaries_rgb = mark_boundaries(
            base, segments, mode=boundary_mode, color=boundary_rgb_color
)
        # Reattach alpha channel if original had it
        if alpha_channel is not None:
            with_boundaries = np.concatenate([with_boundaries_rgb, alpha_channel], axis=-1)
        else:
            with_boundaries = with_boundaries_rgb
        # Boundaries-only view
        boundaries_mask = find_boundaries(segments, mode='outer')
        boundaries_only_rgb = np.zeros_like(rgb_for_slic)
        for c in range(3):
            boundaries_only_rgb[..., c] = boundaries_mask * boundary_rgb_color[c]
        if alpha_channel is not None:
            # transparent background, opaque boundaries
            boundaries_alpha = boundaries_mask.astype(float)
            boundaries_only = np.concatenate([boundaries_only_rgb, boundaries_alpha[..., None]], axis=-1)
        else:
            boundaries_only = boundaries_only_rgb
        overlay_imgs.append(with_boundaries)
        boundary_imgs.append(boundaries_only)
        titles.append(key)

    n = len(overlay_imgs)
    if n == 0:
        fig, ax = plt.subplots(1, 1, figsize=(6, 4))
        ax.axis('off')
        ax.set_title('No images found')
        plt.show()
        return

    fig, axes = plt.subplots(2, n, figsize=(7*n, 10), squeeze=False)
    for i in range(n):
        ax1 = axes[0, i]
        ax1.imshow(overlay_imgs[i])
        ax1.set_title(f"{titles[i]} overlay (n={n_segments}, c={compactness:g}, s={sigma:g})")
        ax1.axis('off')
        ax2 = axes[1, i]
        ax2.imshow(boundary_imgs[i])
        ax2.set_title(f"{titles[i]} boundaries")
        ax2.axis('off')
    plt.tight_layout()
    plt.show()

ui = interactive_output(
    compute_and_show,
    {
        'n_segments': n_segments_slider,
        'compactness': compactness_slider,
        'sigma': sigma_slider,
        'enforce_connectivity': enforce_checkbox,
        'start_label': start_label_selector,
        'boundary_mode': boundary_mode_dropdown,
        'boundary_color': boundary_color_dropdown,
        'fill': fill_checkbox,
        'fill_alpha': fill_alpha_slider,
    }
)

print('Adjust controls below; output updates automatically (RGBA + boundaries-only supported):')
display(controls, ui)

Adjust controls below; output updates automatically (RGBA + boundaries-only supported):


Output()